# Regression with Keras

[link](https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/)

Libraries:

In [64]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

Define Dataset:

In [65]:
X = np.array([[1,2], [3,4], [5,6], [7,8], [9,10]])
Y = np.array([[3], [4], [5], [6], [7]])

Normalize Dataset, if not, the algorithm won't converge!

In [66]:
sc_X = StandardScaler()
X = sc_X.fit_transform(X)

# this is not required
# sc_Y = StandardScaler()
# Y = sc_Y.fit_transform(Y)

## Create the model

In [67]:
def fcn_model():
    model=Sequential()
    model.add(Dense(5, input_dim=2, activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(1))
    
    #Compile the model
    model.compile(loss='mean_squared_error',optimizer='adam')
    
    return model

## Method 1: By using `KerasRegressor`

Using `KerasRegressor` is preferred when we want to evaluate the model via `cross_val_score`.

In [68]:
estimator = KerasRegressor(build_fn=fcn_model, epochs=300, batch_size=5, verbose=0)

1. To define the `Sequential` model, you could alternatively do:
```python
model = Sequential([
    Dense(units=5, input_shape=(2,), activation='relu'),
    Dense(units=5, activation='relu'),
    Dense(1),
])
```

2. Instead of defining function `fcn_model`, we can do:
```python
model=Sequential()
model.add(Dense(5, input_dim=2, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error',optimizer='adam')
estimator = KerasRegressor(build_fn=lambda: model, epochs=300, batch_size=5, verbose=0)
```

## To get rid of warning for `tf.function` retracing:

In [31]:
tf.get_logger().setLevel('ERROR')

## Evaluate the model

In [69]:
kfold = KFold(n_splits=2)
print(cross_val_score(estimator, X, Y, cv=kfold, scoring='neg_mean_squared_error'))

[-10.0504787  -39.40644809]


**Note:** We can not use `model` instead of `estimator` in `cross_val_score` function since thre `model` is not an estimator. (All built-in estimators also have a **`set_params`** method, which sets data-independent parameters) 

In [8]:
estimator.get_params()

{'epochs': 300,
 'batch_size': 5,
 'verbose': 0,
 'build_fn': <function __main__.fcn_model()>}

In [9]:
fcn_model().get_params()

AttributeError: 'Sequential' object has no attribute 'get_params'

## To predict, we need to `.fit()` again:

The fitting will be done inside the `cross_val_score` function, you don't need to worry about this beforehand.

If, besides cross validation, you want to train a model, you can call `model.fit()` afterwards.

In [70]:
estimator.fit(X, Y, epochs=300, batch_size=5, verbose=0)
prediction = estimator.predict(X)

In [71]:
print(Y)
print(prediction)

[[3]
 [4]
 [5]
 [6]
 [7]]
[0.27051312 0.27051312 2.040389   3.3258193  4.631702  ]


### Interesting - 2 observations:

1. Using `lambda` inline function instead of `fnc_model` gives worse result!
2. Running all cells in one cell gives better result!

In [84]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf
import numpy as ny
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

X = ny.array([[1,2], [3,4], [5,6], [7,8], [9,10]])
Y = ny.array([[3], [4], [5], [6], [7]])

sc_X = StandardScaler()
X = sc_X.fit_transform(X)

def fnc_model():
    model=Sequential()
    model.add(Dense(5, input_dim=2, activation='relu'))
    model.add(Dense(5, activation='relu'))
    model.add(Dense(1))

    #Compile the model
    model.compile(loss='mean_squared_error',optimizer='adam')

    return model

estimator = KerasRegressor(build_fn=lambda: model, epochs=300, batch_size=5, verbose=0)
kfold = KFold(n_splits=2)
print(cross_val_score(estimator, X, Y, cv=kfold, scoring='neg_mean_squared_error'))
estimator.fit(X, Y, epochs=300, batch_size=5, verbose=0)
prediction = estimator.predict(X)
print(Y)
print(prediction)

[-0.00618585 -0.00685854]
[[3]
 [4]
 [5]
 [6]
 [7]]
[3.0117946 3.9801488 5.0043645 6.0121584 6.9929504]


In [85]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
import tensorflow as tf
import numpy as ny
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

X = ny.array([[1,2], [3,4], [5,6], [7,8], [9,10]])
Y = ny.array([[3], [4], [5], [6], [7]])

sc_X = StandardScaler()
X = sc_X.fit_transform(X)

model=Sequential()
model.add(Dense(5, input_dim=2, activation='relu'))
model.add(Dense(5, activation='relu'))
model.add(Dense(1))

#Compile the model
model.compile(loss='mean_squared_error',optimizer='adam')

estimator = KerasRegressor(build_fn=lambda: model, epochs=300, batch_size=5, verbose=0)
kfold = KFold(n_splits=2)
print(cross_val_score(estimator, X, Y, cv=kfold, scoring='neg_mean_squared_error'))
estimator.fit(X, Y, epochs=300, batch_size=5, verbose=0)
prediction = estimator.predict(X)
print(Y)
print(prediction)

[-12.069893    -0.83921232]
[[3]
 [4]
 [5]
 [6]
 [7]]
[3.4267864 3.533599  4.169761  5.865515  7.545754 ]


## Method 2: Without using `KerasRegressor`

In [12]:
model = fcn_model()

In [13]:
model.fit(X, Y, batch_size=5, epochs=1000, verbose=0)

In [14]:
prediction = model(X) # may get warning when using prediction = model.predict(X)

**Note:** We _may_ get warning when using `prediction = estimator.predict(X)`. So, we instead use `prediction = estimator.predict(X)`.

In [15]:
print(Y)
print(prediction)

[[3]
 [4]
 [5]
 [6]
 [7]]
tf.Tensor(
[[2.988811]
 [4.029836]
 [4.923675]
 [5.787898]
 [7.291502]], shape=(5, 1), dtype=float32)
